In [7]:
import re
import pandas as pd

path = r'D:\PhD\RA\Schafer\IRA\data\transformed\all_complete.xlsx'
output_path = r'D:\PhD\RA\Schafer\IRA\data\transformed\indications_count.xlsx'

df = pd.read_excel(path)

In [8]:
mask = (
    df['eventtype'].astype(str).str.contains('Approval', case=False, na=False)
    & df['eventdetails'].astype(str).str.contains(r'\bThe US\b', case=False, na=False)
)

def count_by_semicolon(s: str) -> int:
    if not isinstance(s, str):
        return 0
    # strip trailing ; and spaces
    s = s.rstrip('; ').strip()
    # count semicolons -> number of items = semicolons + 1
    return s.count(';')-1

df['indication_count'] = 0
df.loc[mask, 'indication_count'] = df.loc[mask, 'eventdetails'].apply(count_by_semicolon)

df.to_excel(output_path, index=False)

In [6]:
import os, re
import pandas as pd
from fuzzywuzzy import fuzz
from collections import defaultdict

# ---------------- PATHS ----------------
path = r'D:\PhD\RA\Schafer\IRA\data\transformed\indications_count.xlsx'   # must have: drugid, drugprimaryname, origin, nce
partd_path = r"D:\PhD\RA\Schafer\IRA\data\unzipped\merge\processed\partd_combined.xlsx"

out_matches = r"D:\PhD\RA\Schafer\IRA\data\unzipped\merge\processed\partd_matches_with_drugid.xlsx"
out_spend_by_drugid = r"D:\PhD\RA\Schafer\IRA\data\unzipped\merge\processed\partd_spend_by_drugid.xlsx"
out_panel   = r"D:\PhD\RA\Schafer\IRA\data\transformed\indications_with_partd_totals_by_drugid.xlsx"

for p in [out_matches, out_spend_by_drugid, out_panel]:
    os.makedirs(os.path.dirname(p), exist_ok=True)

# ---------------- HELPERS ----------------
def normalize(s):
    """Keep + and / so we can tokenize combos; collapse other punctuation to spaces."""
    if pd.isna(s): return ""
    s = str(s).lower().strip().replace("\u00a0", " ")
    s = re.sub(r"[-_,;:()]+", " ", s)  # DO NOT remove + or /
    s = re.sub(r"\s+", " ", s).strip()
    return s

def tokenize_for_count(s: str):
    # split on +, /, space, hyphen; keep non-empty lower tokens
    return [t for t in re.split(r"[+/ \-]+", s.lower().strip()) if t]

def tokset(s: str):
    return set(tokenize_for_count(s))

def token_count(s: str) -> int:
    return len(tokset(s))

# ---------------- LOAD ----------------
df = pd.read_excel(path)          # must have: 'drugid','drugprimaryname','origin','nce'
partd = pd.read_excel(partd_path) # must contain: Gnrc_Name (and Tot_Spndng_*)

# ----- flags: new_bio, new_small -----
origin_str = df["origin"].astype(str).str.lower()
df["new_bio"] = (
    origin_str.str.contains("protein", na=False) &
    (~origin_str.str.contains(r"bio\s*-?\s*similar", regex=True, na=False))
).astype(int)

nce_numeric = pd.to_numeric(df["nce"], errors="coerce")
df["new_small"] = ((nce_numeric == 1) & (df["new_bio"] == 0)).astype(int)

# ---------------- PREP (global) ----------------
df = df.copy()
df["primary_clean"]  = df["drugprimaryname"].map(normalize)
df["primary_tok_ct"] = df["primary_clean"].map(token_count)
df["primary_tokset"] = df["primary_clean"].map(tokset)
primary_to_name = dict(zip(df["primary_clean"], df["drugprimaryname"]))

partd = partd.copy().reset_index(drop=True)
partd["partd_row_id"]    = partd.index
partd["Gnrc_Name_clean"] = partd["Gnrc_Name"].map(normalize)
partd["gnrc_tok_ct"]     = partd["Gnrc_Name_clean"].map(token_count)
partd["gnrc_tokset"]     = partd["Gnrc_Name_clean"].map(tokset)

# spend columns
spend_cols = [c for c in partd.columns if str(c).startswith("Tot_Spndng_")]
if spend_cols:
    partd[spend_cols] = (
        partd[spend_cols]
        .apply(lambda s: s.astype(str)
               .str.replace(r"[,\$]", "", regex=True)
               .str.replace(r"[—–]+", "", regex=True)
               .str.replace(r"(?i)suppressed|n/?a|\*|nan|none", "", regex=True)
               .str.strip())
        .apply(pd.to_numeric, errors="coerce")
    )

# ---------------- MATCHING CORE (no token-count gating) ----------------
SIMPLE_CUTOFF = 79   # fuzzy cutoff

def run_pass(partd_remaining: pd.DataFrame, df_pool: pd.DataFrame, pass_tag: str):
    """
    Exact match, then fuzzy WITHOUT token-count restrictions:
      - Candidates share ≥1 token with the Part D name
      - Choose by MAX shared tokens; tie-break by fuzzy score
    Returns only the hits from this pass.
    """
    if df_pool.empty or partd_remaining.empty:
        return pd.DataFrame(columns=["partd_row_id","Gnrc_Name","Gnrc_Name_clean",
                                     "matched_primary_clean","match_type","match_score"] + spend_cols)

    # Build token index for this pool
    primary_set_map  = dict(zip(df_pool["primary_clean"], df_pool["primary_tokset"]))
    token_to_primary = defaultdict(set)
    for pc, pset in primary_set_map.items():
        for t in pset:
            token_to_primary[t].add(pc)

    # 1) exact within pool
    exact = partd_remaining.merge(
        df_pool[["primary_clean"]].drop_duplicates(),
        left_on="Gnrc_Name_clean",
        right_on="primary_clean",
        how="left"
    ).rename(columns={"primary_clean":"matched_primary_clean"})

    exact["match_type"]  = exact["matched_primary_clean"].notna().map({True: "generic_exact", False: None})
    exact["match_score"] = exact["matched_primary_clean"].notna().map({True: 100, False: None})

    exact_hits   = exact[exact["matched_primary_clean"].notna()].copy()
    matched_exact_ids = set(exact_hits["partd_row_id"])

    # 2) fuzzy for the remaining Part D rows — NO token-count tolerance, only token overlap
    rem = partd_remaining.loc[~partd_remaining["partd_row_id"].isin(matched_exact_ids),
                              ["partd_row_id","Gnrc_Name","Gnrc_Name_clean","gnrc_tok_ct","gnrc_tokset"] + spend_cols].copy()

    rows = []
    for _, r in rem.iterrows():
        gset = set(r["gnrc_tokset"])
        if not gset:
            continue

        # candidates = any primary sharing ≥1 token
        cand_primaries = set()
        for t in gset:
            cand_primaries |= token_to_primary.get(t, set())
        if not cand_primaries:
            continue

        best_pc, best_shared, best_score = None, -1, 0
        for pc in cand_primaries:
            pset = primary_set_map[pc]
            shared_ct = len(gset & pset)
            sc = max(
                fuzz.token_set_ratio(r["Gnrc_Name_clean"], pc),
                fuzz.token_sort_ratio(r["Gnrc_Name_clean"], pc)
            )
            if sc < SIMPLE_CUTOFF:
                continue
            if (shared_ct > best_shared) or (shared_ct == best_shared and sc > best_score):
                best_pc, best_shared, best_score = pc, shared_ct, sc

        if best_pc is not None:
            row = {
                "partd_row_id": r["partd_row_id"],
                "Gnrc_Name": r["Gnrc_Name"],
                "Gnrc_Name_clean": r["Gnrc_Name_clean"],
                "matched_primary_clean": best_pc,
                "match_type": "generic_simple",
                "match_score": best_score,
            }
            for c in spend_cols:
                row[c] = r[c]
            rows.append(row)

    simple_hits = pd.DataFrame(rows) if rows else pd.DataFrame(
        columns=["partd_row_id","Gnrc_Name","Gnrc_Name_clean",
                 "matched_primary_clean","match_type","match_score"] + spend_cols
    )

    # combine
    hits = pd.concat(
        [
            exact_hits[["partd_row_id","Gnrc_Name","Gnrc_Name_clean",
                        "matched_primary_clean","match_type","match_score"] + spend_cols],
            simple_hits[["partd_row_id","Gnrc_Name","Gnrc_Name_clean",
                         "matched_primary_clean","match_type","match_score"] + spend_cols],
        ],
        ignore_index=True
    )
    return hits

# ---------------- TWO-PASS MATCHING ----------------
# Pass 1: PRIORITY pool (new_small==1 OR new_bio==1)
priority_pool = df.loc[(df["new_small"]==1) | (df["new_bio"]==1)].copy()
general_pool  = df.loc[~df.index.isin(priority_pool.index)].copy()

partd_remaining = partd.copy()

pass1_hits = run_pass(partd_remaining, priority_pool, pass_tag="priority")
matched_ids_pass1 = set(pass1_hits["partd_row_id"])

# remove matched Part D rows before Pass 2
partd_remaining = partd_remaining.loc[~partd_remaining["partd_row_id"].isin(matched_ids_pass1)].copy()

# Pass 2: GENERAL pool (leftover DF)
pass2_hits = run_pass(partd_remaining, general_pool, pass_tag="general")
matched_ids_pass2 = set(pass2_hits["partd_row_id"])

# ---------------- STEP 3: TOKEN-CONTAINMENT (UNMATCHED indications vs leftover Part D) ----------------
# Indications already matched in Pass 1+2:
matched_primary_pass12 = set(
    pd.concat([pass1_hits["matched_primary_clean"], pass2_hits["matched_primary_clean"]]).dropna().unique()
)

# Candidate indications for Step 3 = UNMATCHED only
df_unmatched_pool = df.loc[~df["primary_clean"].isin(matched_primary_pass12)].copy()

# Leftover Part D rows after Pass 2
leftover = partd_remaining.loc[~partd_remaining["partd_row_id"].isin(matched_ids_pass2)].copy()

def token_containment_pass(partd_leftover: pd.DataFrame, df_candidates: pd.DataFrame):
    if partd_leftover.empty or df_candidates.empty:
        return pd.DataFrame(columns=["partd_row_id","Gnrc_Name","Gnrc_Name_clean",
                                     "matched_primary_clean","match_type","match_score"] + spend_cols)

    # Map token -> candidate primary_clean
    token_to_primary = defaultdict(set)
    primary_set_map  = dict(zip(df_candidates["primary_clean"], df_candidates["primary_tokset"]))
    for pc, pset in primary_set_map.items():
        for t in pset:
            token_to_primary[t].add(pc)

    rows = []
    for _, r in partd_leftover.iterrows():
        gset = set(r["gnrc_tokset"])
        cand_primaries = set()
        for t in gset:
            cand_primaries |= token_to_primary.get(t, set())
        if not cand_primaries:
            continue

        best_pc, best_shared, best_score = None, -1, 0
        for pc in cand_primaries:
            pset = primary_set_map[pc]
            shared = len(gset & pset)
            if shared == 0:
                continue
            sc = fuzz.token_set_ratio(r["Gnrc_Name_clean"], pc)
            if (shared > best_shared) or (shared == best_shared and sc > best_score):
                best_pc, best_shared, best_score = pc, shared, sc

        if best_pc is not None:
            row = {
                "partd_row_id": r["partd_row_id"],
                "Gnrc_Name": r["Gnrc_Name"],
                "Gnrc_Name_clean": r["Gnrc_Name_clean"],
                "matched_primary_clean": best_pc,
                "match_type": "token_contains",
                "match_score": best_score,
            }
            for c in spend_cols:
                row[c] = r[c]
            rows.append(row)

    return pd.DataFrame(rows)

token_hits = token_containment_pass(leftover, df_unmatched_pool)

# ---------------- COMBINE ALL MATCHES ----------------
matches = pd.concat([pass1_hits, pass2_hits, token_hits], ignore_index=True)

# ---------------- ATTACH drugid ----------------
matches = matches.merge(
    df[["primary_clean","drugid"]].drop_duplicates(),
    left_on="matched_primary_clean",
    right_on="primary_clean",
    how="left"
).drop(columns=["primary_clean"])

matches["matched_primary_name"] = matches["matched_primary_clean"].map(primary_to_name)
matches.to_excel(out_matches, index=False)

# ---------------- SUM spending BY drugid ----------------
if spend_cols:
    spend_by_drugid = (
        matches.dropna(subset=["drugid"])[["drugid"] + spend_cols]
               .groupby("drugid", as_index=False)
               .sum(min_count=1)
    )
else:
    spend_by_drugid = pd.DataFrame(columns=["drugid"])
spend_by_drugid.to_excel(out_spend_by_drugid, index=False)

# ---------------- MERGE totals back to your panel by drugid ----------------
df_with_totals = df.merge(spend_by_drugid, on="drugid", how="left")
df_with_totals.to_excel(out_panel, index=False)

# ---------- Quick QA ----------
exact_ct_1   = (pass1_hits["match_type"] == "generic_exact").sum()
simple_ct_1  = (pass1_hits["match_type"] == "generic_simple").sum()
exact_ct_2   = (pass2_hits["match_type"] == "generic_exact").sum()
simple_ct_2  = (pass2_hits["match_type"] == "generic_simple").sum()
token_ct     = (token_hits["match_type"] == "token_contains").sum()

matched_ids_all = pd.concat(
    [pass1_hits["partd_row_id"], pass2_hits["partd_row_id"], token_hits["partd_row_id"]]
).nunique()
unmatched_ct = len(partd) - matched_ids_all
mapped_ids   = matches["drugid"].nunique(dropna=True)

print("---- PASS 1 (priority: new_small/new_bio) ----")
print(f"Exact matches: {exact_ct_1:,}")
print(f"Simple matches (score>=79 & max shared tokens): {simple_ct_1:,}")
print("---- PASS 2 (general leftovers) ----")
print(f"Exact matches: {exact_ct_2:,}")
print(f"Simple matches (score>=79 & max shared tokens): {simple_ct_2:,}")
print("---- PASS 3 (token containment, UNMATCHED indications) ----")
print(f"Token-contained matches: {token_ct:,}")
print("-------------------------------------")
print(f"Unmatched Part D rows: {unmatched_ct:,}")
print(f"Mapped unique drugids: {mapped_ids:,}")
print(f"Final panel written to: {out_panel}")


---- PASS 1 (priority: new_small/new_bio) ----
Exact matches: 934
Simple matches (score>=79 & max shared tokens): 1,234
---- PASS 2 (general leftovers) ----
Exact matches: 85
Simple matches (score>=79 & max shared tokens): 944
---- PASS 3 (token containment, UNMATCHED indications) ----
Token-contained matches: 1,098
-------------------------------------
Unmatched Part D rows: 409
Mapped unique drugids: 1,411
Final panel written to: D:\PhD\RA\Schafer\IRA\data\transformed\indications_with_partd_totals_by_drugid.xlsx


In [7]:
import re
import pandas as pd

# ---------------- PATHS ----------------
path = r"D:\PhD\RA\Schafer\IRA\data\transformed\indications_with_partd_totals_by_drugid.xlsx"
output_path = r"D:\PhD\RA\Schafer\IRA\data\transformed\indications_partd_approval.xlsx"

# ---------------- LOAD ----------------
df = pd.read_excel(path)

# ---------------- COLUMN NAMES ----------------
drug_col = "drugid"
etype_col = "eventtype"
edet_col  = "eventdetails"
date_col  = "eventdate"   # you said the date column is eventdate

# ---------------- NORMALIZE DATE ----------------
df[date_col] = pd.to_datetime(df[date_col], errors="coerce")

# ---------------- "US" DETECTOR ----------------
# Exact “The US” (case-insensitive). If you want to be more flexible, see the alt regex below.
US_RE = re.compile(r"\bThe\s*US\b", flags=re.IGNORECASE)
# Flexible alternative (comment the line above and uncomment the line below if desired):
# US_RE = re.compile(r"\b(?:The\s*US|US|U\.S\.A?|USA|United\s+States)\b", re.IGNORECASE)

# ---------------- INDICATION COUNT (AFTER "The US") ----------------
def count_after_us_items(text) -> int:
    """
    After 'The US', strip leading/trailing separators.
    If tail is non-empty and has NO semicolons -> return 1.
    Otherwise -> return the count of semicolons.
    """
    if not isinstance(text, str):
        return 0

    m = US_RE.search(text)
    if not m:
        return 0

    # tail after 'The US'
    tail = text[m.end():]

    # strip leading and trailing separators (incl. leading/trailing ';')
    tail = re.sub(r'^[\s:–—\-;]+', '', tail)   # leading
    tail = re.sub(r'[\s;.,]+$', '', tail)      # trailing

    if not tail:
        return 0

    semi_count = tail.count(';')
    return 1 if semi_count == 0 else semi_count


mask_us   = df[edet_col].astype(str).str.contains(US_RE, na=False)
mask_appr = df[etype_col].astype(str).str.contains("Approval", case=False, na=False)

# create/overwrite indication_count
df["indication_count"] = 0
df.loc[mask_us & mask_appr, "indication_count"] = (
    df.loc[mask_us & mask_appr, edet_col].apply(count_after_us_items)
)

# ---------------- FIRST DATES BY DRUGID (US-only) ----------------
# Approval and Launch masks
mask_launch = df[etype_col].astype(str).str.contains("Launch", case=False, na=False)

# Earliest US+Approval per drug
fa_by_id = (
    df.loc[mask_us & mask_appr, [drug_col, date_col]]
      .groupby(drug_col, dropna=False)[date_col]
      .min()
      .rename("first_approval_date")
      .reset_index()
)

# Earliest US+Launch per drug
fl_by_id = (
    df.loc[mask_us & mask_launch, [drug_col, date_col]]
      .groupby(drug_col, dropna=False)[date_col]
      .min()
      .rename("first_launch_date")
      .reset_index()
)

# Merge back to main df
for col in ["first_approval_date", "first_launch_date", "first_approval_or_launch"]:
    if col not in df.columns:
        df[col] = pd.NaT

df = df.drop(columns=["first_approval_date", "first_launch_date", "first_approval_or_launch"], errors="ignore")
df = df.merge(fa_by_id, on=drug_col, how="left")
df = df.merge(fl_by_id, on=drug_col, how="left")

# approval-or-launch preference: approval if present, else launch
df["first_approval_or_launch"] = df["first_approval_date"].where(
    df["first_approval_date"].notna(),
    df["first_launch_date"]
)

# ---------------- SAVE ----------------
df.to_excel(output_path, index=False)


In [8]:
import re
import pandas as pd

# ---------------- PATHS ----------------
path = r"D:\PhD\RA\Schafer\IRA\data\transformed\indications_partd_approval.xlsx"
output_path = r"D:\PhD\RA\Schafer\IRA\data\transformed\indications_partd_approval_trial.xlsx"

# ---------------- LOAD ----------------
df = pd.read_excel(path)

# ---------------- COL GUARDS ----------------
# handle occasional misspelling 'eventdetials'
details_col = "eventdetails"
type_col = "eventtype"

#base mask (no capturing groups)
base_mask = df[type_col].astype(str).str.contains(
    r'(?:Global Status Advance|Disease Phase Change)', case=False, na=False, regex=True
)

# compile patterns with NON-capturing groups
NDR_RE       = re.compile(r'\b(?:NDR|No\s*Devel)\b', re.IGNORECASE)
TRIAL_RE     = re.compile(r'\btrials?\b', re.IGNORECASE)  # no group needed

PHASE_I_RE   = re.compile(r'\b(?:ph(?:ase)?\s*i|ph(?:ase)?\s*1)\b',   re.IGNORECASE)
PHASE_II_RE  = re.compile(r'\b(?:ph(?:ase)?\s*ii|ph(?:ase)?\s*2)\b',  re.IGNORECASE)
PHASE_III_RE = re.compile(r'\b(?:ph(?:ase)?\s*iii|ph(?:ase)?\s*3)\b', re.IGNORECASE)

det = df[details_col].astype(str)

df["NDR"]      = (base_mask & det.str.contains(NDR_RE,       na=False)).astype(int)
df["trial"]    = (base_mask & det.str.contains(TRIAL_RE,     na=False)).astype(int)
df["phasei"]   = (base_mask & det.str.contains(PHASE_I_RE,   na=False)).astype(int)
df["phaseii"]  = (base_mask & det.str.contains(PHASE_II_RE,  na=False)).astype(int)
df["phaseiii"] = (base_mask & det.str.contains(PHASE_III_RE, na=False)).astype(int)

# ---------------- COLLAPSE LOGIC ----------------
# If Phase III is present, zero out Phase I and Phase II (since Phase III implies earlier phases)
df.loc[df["phaseiii"] == 1, ["phasei", "phaseii"]] = 0
df.loc[df["phaseii"] == 1, ["phasei"]] = 0

df["other"] = 0
bucket_cols = ["NDR", "trial", "phasei", "phaseii", "phaseiii"]
df.loc[base_mask & (df[bucket_cols].sum(axis=1) == 0), "other"] = 1

phase_cols = ["phasei", "phaseii", "phaseiii"]
df["trial"] = ((df["trial"] == 1) | df[phase_cols].any(axis=1)).astype(int)

In [9]:
date_col = "eventdate"

# --- datetimes & sort ---
df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
df["first_approval_date"] = pd.to_datetime(df["first_approval_date"], errors="coerce")
df = df.sort_values(["drugid", date_col], kind="mergesort")

# --- use the existing first_approval_date per drugid and broadcast it ---
# (assumes exactly one non-null per drugid; transform('max') copies that value to all rows)
anchor = df.groupby("drugid")["first_approval_date"].transform("max")

# --- rows to count in cumulatives: on/after anchor & anchor exists ---
df["counting_mask"] = anchor.notna() & (df[date_col] >= anchor)

# --- any phase at row ---
df["phase_any"] = ((df["phasei"] == 1) | (df["phaseii"] == 1) | (df["phaseiii"] == 1)).astype(int)

# --- cumulative helper (within each drugid) ---
def _cum_from_anchor(g: pd.DataFrame, col: str) -> pd.Series:
    vals = g[col].fillna(0).where(g["counting_mask"], 0)
    return vals.cumsum()

# cumulative indication_count (from first approval onward)
df["cum_indications"] = (
    df.groupby("drugid", group_keys=False)
      .apply(_cum_from_anchor, col="indication_count")
)

# cumulative trials
df["cum_trials"] = (
    df.groupby("drugid", group_keys=False)
      .apply(_cum_from_anchor, col="trial")
)

# cumulative phases (any phase)
df["cum_phases"] = (
    df.groupby("drugid", group_keys=False)
      .apply(_cum_from_anchor, col="phase_any")
)

# (Optional) per-phase cumulatives
df["cum_phasei"]   = df.groupby("drugid", group_keys=False).apply(_cum_from_anchor, col="phasei")
df["cum_phaseii"]  = df.groupby("drugid", group_keys=False).apply(_cum_from_anchor, col="phaseii")
df["cum_phaseiii"] = df.groupby("drugid", group_keys=False).apply(_cum_from_anchor, col="phaseiii")

# flag: drug has a non-empty first_approval_date
df["has_first_approval_date"] = anchor.notna().astype(int)

# SAVE
df.to_excel(output_path, index=False)

C:\Users\yizho\AppData\Local\Temp\ipykernel_1412\1847737857.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_cum_from_anchor, col="indication_count")
C:\Users\yizho\AppData\Local\Temp\ipykernel_1412\1847737857.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_cum_from_anchor, col="trial")
C:\Users\yizho\AppData\Local\Temp\ipykernel_1412\1847737857.py:38: DeprecationWarning: DataFrameG

In [10]:
import pandas as pd
import numpy as np

# ---------------- PATHS ----------------
path = r"D:\PhD\RA\Schafer\IRA\data\transformed\indications_partd_approval_trial.xlsx"
output_path = r"D:\PhD\RA\Schafer\IRA\data\transformed\indications_partd_approval_trial_yearly.xlsx"

# ---------------- LOAD ----------------
df = pd.read_excel(path)

# ---------------- DATES ----------------
date_col = "eventdate"
if date_col not in df.columns:
    raise KeyError(f"Missing required date column: {date_col}")

df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
df["first_approval_date"] = pd.to_datetime(df.get("first_approval_date"), errors="coerce")

# ---------------- INDICATIONS @ APPROVAL (per drug) ----------------
# anchor per row = existing first_approval_date for that drug (broadcast)
anchor = df.groupby("drugid")["first_approval_date"].transform("max")

# mark rows that fall on the first approval date (compare by date, ignore time)
is_app_day = anchor.notna() & (df[date_col].dt.normalize() == anchor.dt.normalize())

# sum indication_count on the approval date per drugid, then merge back
if "indication_count" not in df.columns:
    raise KeyError("Missing required column: indication_count")

ind_at_app = (
    df.loc[is_app_day, ["drugid", "indication_count"]]
      .groupby("drugid", as_index=False)["indication_count"].sum()
      .rename(columns={"indication_count": "indications_at_app"})
)

df = df.merge(ind_at_app, on="drugid", how="left")
df["indications_at_app"] = df["indications_at_app"].fillna(0).astype(int)

# ---------------- YEAR + APPYEAR ----------------
df["year"] = df[date_col].dt.year
# ensure datetime then derive appyear
df["first_approval_date"] = pd.to_datetime(df.get("first_approval_date"), errors="coerce")
df["appyear"] = df["first_approval_date"].dt.year

# ---------------- LAST OBSERVATION PER (drugid, year) ----------------
# Sort then keep the last row within each (drugid, year)
df_sorted = df.sort_values(["drugid", "year", date_col], kind="mergesort")
year_last = df_sorted.drop_duplicates(subset=["drugid", "year"], keep="last").copy()

# ---------------- EXPAND YEARS FROM APPYEAR TO 2025 ----------------
END_YEAR = 2025

# We only expand for drugids with a non-null appyear that is <= END_YEAR
base = year_last.copy()
valid = base.dropna(subset=["appyear"]).query("appyear <= @END_YEAR")[["drugid", "appyear"]].drop_duplicates()

# Build a full grid of (drugid, year) from appyear..END_YEAR for each valid drug
grids = []
for rid, appy in valid.itertuples(index=False):
    start = int(appy)
    if start > END_YEAR:
        continue
    yrs = np.arange(start, END_YEAR + 1, dtype=int)
    grids.append(pd.DataFrame({"drugid": rid, "year": yrs, "appyear": start}))
full_grid = pd.concat(grids, ignore_index=True) if grids else pd.DataFrame(columns=["drugid","year","appyear"])

# Merge grid with the last-obs rows; then forward-fill within each drugid by year
year_panel = full_grid.merge(base, on=["drugid", "year"], how="left", suffixes=("", "_orig"))

# Ensure rows are ordered to ffill correctly
year_panel = year_panel.sort_values(["drugid", "year"], kind="mergesort")

# Columns we should forward fill. By default, carry forward *everything* except the keys and raw eventdate.
# If you want to limit to specific columns, list them explicitly instead.
dont_ffill = {"drugid", "year", "appyear"}  # keep appyear as constant; it's already present from the grid
if date_col in year_panel.columns:
    dont_ffill.add(date_col)

ffill_cols = [c for c in year_panel.columns if c not in dont_ffill]

year_panel[ffill_cols] = (
    year_panel.groupby("drugid", as_index=False)[ffill_cols]
              .ffill()
)

# For drugids without any appyear (NaN), keep their observed (drugid, year) rows as-is
# Append non-expanded drugs (if any)
no_app = base.loc[base["appyear"].isna(), :].copy()
if not no_app.empty:
    # We already kept last obs per (drugid, year) in `base`; just append those rows
    # (no expansion since appyear is missing)
    keep_cols = year_panel.columns
    no_app = no_app.reindex(columns=keep_cols)
    year_panel = pd.concat([year_panel, no_app], ignore_index=True)

# Sort final panel
year_panel = year_panel.sort_values(["drugid", "year"], kind="mergesort").reset_index(drop=True)

# Optional: make sure types look good
year_panel["indications_at_app"] = year_panel["indications_at_app"].fillna(0).astype(int)
# If you have cumulative cols (e.g., cum_indications, cum_trials…), they’ll be carried forward as well.



# 1) Identify and coerce Part D spend columns
spend_cols = [c for c in year_panel.columns if str(c).startswith("Tot_Spndng_")]
if spend_cols:
    year_panel[spend_cols] = year_panel[spend_cols].apply(pd.to_numeric, errors="coerce")

# 2) Per-drug 2021 spend and rank (higher spend -> rank 1)
spend_2021_col = "Tot_Spndng_2021"

drug_spend_2021 = (
    year_panel.groupby("drugid", as_index=False)[spend_2021_col]
              .max()  # same within drug; max ignores NaN
              .rename(columns={spend_2021_col: "spend_2021"})
)

drug_spend_2021["rank_spend_2021"] = (
    drug_spend_2021["spend_2021"]
        .rank(method="dense", ascending=False, na_option="bottom")
        .astype("Int64")
)

# 3) Merge ranks back to every row of that drug
year_panel = year_panel.merge(
    drug_spend_2021[["drugid", "spend_2021", "rank_spend_2021"]],
    on="drugid", how="left"
)

# 4) Top-N dummies
year_panel["top50"]  = year_panel["rank_spend_2021"].le(50).astype("Int64")
year_panel["top100"] = year_panel["rank_spend_2021"].le(100).astype("Int64")
year_panel["top250"] = year_panel["rank_spend_2021"].le(250).astype("Int64")

# 5) partd flag = 1 if ANY Part D spend column is non-empty for this drug (broadcast to all its rows)
if spend_cols:
    row_has_spend = year_panel[spend_cols].notna().any(axis=1)
    year_panel["partd"] = row_has_spend.groupby(year_panel["drugid"]).transform("max").astype(int)
else:
    year_panel["partd"] = 0
    
# ---------------- SAVE ----------------
year_panel.to_excel(output_path, index=False)